In [ ]:
# Mount drive to work in google colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt

from math import floor

# Path to data folder
path_to_data = 'data'

target = 'catalao_A034.csv'
all_data_file = 'alldata.csv'

In [ ]:
data_all = pd.read_csv(path_to_data+'/preprocessed/'+all_data_file, sep = ';', index_col=0, low_memory=False, parse_dates=True)
data_target = pd.read_csv(path_to_data+'/preprocessed/'+target, sep = ';', index_col=0, low_memory=False, parse_dates=True)

In [ ]:
data_target.columns

In [ ]:
select_columns = [
                    'Precipitação',
                    'Temperatura máxima',
                    'Temperatura mínima',
                    'Umidade relativa máxima',
                    'Umidade relativa mínima',
                    'Vento rajada máxima',
]

data_target = data_target.loc[:, select_columns]

In [ ]:
scaler_all = MinMaxScaler(feature_range = (-1, 1))
scaler_target = MinMaxScaler(feature_range = (-1, 1))

scaled_data_all = scaler_all.fit_transform(data_all)
scaled_data_target = scaler_target.fit_transform(data_target)

In [ ]:
def windowing(data, target, size=2, horizons=[1]):
  X_data  = []
  y_data  = []
  bigger_horizon = horizons[-1]
  correct_horizontal_time = bigger_horizon - 1
  for i in range(size, target.shape[0] - correct_horizontal_time):
    X_data.append(list(data[i-size:i, :].flatten()))
    target_y = np.array([])
    for horizon in horizons:
      target_y = np.concatenate((target_y, target[i+horizon-1, :]), axis=None)
    y_data.append(target_y)

  return np.array(X_data), np.array(y_data)

In [ ]:
len(scaled_data_target[10:12][0])

In [ ]:
window_size = 18
horizons = [1, 2, 3, 4, 5, 6, 12, 24, 48]
data, target = windowing(scaled_data_all, scaled_data_target, window_size, horizons)

In [ ]:
window_size

In [ ]:
data.shape

In [ ]:
target.shape

In [ ]:
data_train, data_test, target_train, target_test = train_test_split(data, target, shuffle=True, train_size=0.8, random_state = 41)

In [ ]:
data_train.shape

In [ ]:
data_test.shape

In [ ]:
target_train.shape

In [ ]:
regressor = Sequential()

regressor.add(Dense(units = 128, input_shape = (data_train.shape[1],)))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 512))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 256))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = target_train.shape[1]))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['mean_absolute_error'])

callbacks = [
    EarlyStopping(monitor='loss', patience=10, restore_best_weights=True),
]

history = regressor.fit(data_train, target_train, epochs = 200, batch_size = 128, shuffle=True, callbacks=callbacks, validation_split=0.1)

In [ ]:
predicted = regressor.predict(data_test)

In [ ]:
predicted.shape

In [ ]:
def separa_horizontes(dados, horizons_size):
  dados_separados = []
  for dado in dados:
    quantidade_por_horizonte = round(len(dado) / horizons_size)
    dado_separado = []
    for horizonte in range(horizons_size):
      inicio = horizonte*quantidade_por_horizonte
      fim = inicio + quantidade_por_horizonte
      dado_separado.append(dado[inicio:fim])
    dados_separados.append(dado_separado)
  return np.array(dados_separados)

In [ ]:
predicted_separeted = separa_horizontes(predicted, len(horizons))
target_test_separeted = separa_horizontes(target_test, len(horizons))

In [ ]:
predicted_separeted.shape

In [ ]:
target_test_separeted.shape

In [ ]:
unnormalized_predicted = []
unnormalized_target = []
for horizon in range(len(horizons)):
  unnormalized_predicted.append(scaler_target.inverse_transform(predicted_separeted[:, horizon, :]))
  unnormalized_target.append(scaler_target.inverse_transform(target_test_separeted[:, horizon, :]))

In [ ]:
column_names = list(data_target.columns)

In [ ]:
for position, horizon in enumerate(horizons):
  target = unnormalized_target[position]
  predicted = unnormalized_predicted[position]
  print(f'Horizon: {horizon}')
  print('MAE')
  for index, column in enumerate(column_names):
    mae = mean_absolute_error(target[:, index],predicted[:, index])
    print(column+': '+str(mae))
  print('\n-------------------------\n')


In [ ]:
 for position, horizon in enumerate(horizons):
  target = unnormalized_target[position]
  predicted = unnormalized_predicted[position]

  target_init = predicted_init = round(target.shape[0] * 0)
  size = 0.05
  target_final = round(target.shape[0] * size)
  predicted_final = round(predicted.shape[0] * size)

  print(f'Horizon: {horizon}')
  for index, column in enumerate(column_names):
    plt.figure(figsize=(30,6))
    plt.plot(target[:target_final, index], color = 'black', label = 'Real')
    plt.plot(predicted[:predicted_final, index], color = 'blue', label = 'Predito')
    plt.title(column)
    plt.xlabel('Time')
    plt.ylabel(column)
    plt.legend()
    plt.show()
  print('\n' + '-'*100 + '\n')